# ①

In [ ]:
import guidance
guidance.llm = guidance.llms.OpenAI("text-davinci-003")

In [ ]:
# プレースホルダを含むテンプレートを定義
program = guidance('''{{user_input}}について教えて。''')

# プロンプトの生成
executed_program = program(
    user_input="三大栄養素"
)

In [ ]:
# {{gen}}コマンドによる動的なプロンプト生成
program = guidance('''{{user_input}}について教えて。 
    {{gen 'generated' n=1 temperature=0.7 max_tokens=64 top_p=0.9}}''')

executed_program = program(user_input="三大栄養素")

In [ ]:
# {{gen}}コマンドによる動的なプロンプト生成②
program = guidance('''{{user_input}}を摂取する上で大事なことは {{gen 'things' n=2 temperature=0.7 max_tokens=32}}''')

executed_program = program(user_input="三大栄養素")

In [ ]:
for thing in executed_program['things']:
    print(thing)

In [ ]:
# テンプレート内で使用したいリストを定義
nutrient_list = ["炭水化物", "タンパク質", "脂質"]

# テンプレート内でリストを展開
program = guidance('''栄養素リスト:
{{#each nutrient_list}}- {{this}}
{{/each}}''')

# プロンプトを生成
executed_program = program(
    nutrient_list=nutrient_list
)

In [ ]:
# テンプレート内で使用したい辞書型リストを定義
people = [
    {"name": "田中",  "description": "鶏むね肉と白米を食べました"},
    {"name": "木村",  "description": "ラーメンと餃子を食べました"}
]

# テンプレート内で辞書型リストを展開
program = guidance('''食事状況:
{{#each people}} {{this.name}}: {{this.description}}
{{/each}}''')

# プロンプトを生成
executed_program = program(
    people=people
)

In [ ]:
# テンプレート内で使用する変数を定義
nutrient_list = ["炭水化物", "タンパク質", "脂質"]
people = [
    {"name": "田中",  "description": "鶏むね肉と白米を食べました"},
    {"name": "木村",  "description": "ラーメンと餃子を食べました"}
]

# プロンプト①を生成
program1 = guidance('''栄養素リスト:
{{#each nutrient_list}}- {{this}}
{{/each~}}''')

# プロンプト②の中でプロンプト①を展開する
program2 = guidance('''{{>program1}}
食事状況:
{{#each people}} {{this.name}}: {{this.description}}
{{/each}}''')

# プロンプトを生成
executed_program = program2(
    program1=program1, 
    nutrient_list=nutrient_list,
    people=people
)

In [ ]:
# テンプレート内で使用する変数を定義
nutrient_list = ["炭水化物", "タンパク質", "脂質"]
people = [
    {"name": "田中",  "description": "鶏むね肉と白米を食べました"},
    {"name": "木村",  "description": "ラーメンと餃子を食べました"}
]

# プロンプト①を生成
program1 = guidance('''栄養素リスト:
{{#each nutrient_list}}- {{this}}
{{/each~}}''')

# プロンプト②を作成
program2 = guidance('''食事状況:
{{#each people}} {{this.name}}: {{this.description}}
{{/each}}
''')

# 指示文を作成 (モデルにやってもらいたいこと)
instruct = guidance('''あなたは食事のアドバイザーです。
以下のJSONフォーマットに従い、ユーザへのアドバイスを作成してください。
アドバイスを考える際は、足りない栄養を考慮する必要があります。

$JSON_BLOB = {
    "name": "ユーザ名",
    "help": "アドバイス内容"
}

以下の栄養素リストと食事状況に基づいたアドバイスを行います。
{{>program1}}
{{>program2}}

アドバイス：{{~gen 'result' n=1 temperature=0.9 max_tokens=256}}''')

In [ ]:
executed_program = instruct(
    # プロンプト①に関する引数
    nutrient_list=nutrient_list,
    program1=program1,
    # プロンプト②に関する引数
    people=people,
    program2=program2
)

In [ ]:
executed_program['result']

In [ ]:
guidance.llms.OpenAI.cache.clear()

# ②

In [ ]:
# Yes, No, Maybeを答えさせる
options = ["Yes", "No", "Maybe"]
policy_list = [
    "メールアドレスが含まれている場合、必ずポリシー違反と判断する。"
]
sentence = "test@gmail.comにメールを送りたい。"

program = guidance('''以下の文章はポリシーに準拠していますか？{{options}}のどれかで回答して。
ポリシー:
{{#each policy_list}}- {{this}}
{{/each}}
文章: {{sentence}}
Answer: {{select "answer" options=options}}''')

executed_program = program(
    policy_list = policy_list,
    sentence = sentence,
    options = options
)

In [ ]:
# 指示を一部非表示にする
options = ["Yes", "No", "Maybe"]
policy_list = [
    "メールアドレスが含まれている場合、必ずポリシー違反と判断する。"
]
sentence = "test@gmail.comにメールを送りたい。"

program = guidance('''以下の文章はポリシーに準拠していますか？
ポリシー:
{{#each policy_list}}- {{this}}
{{/each~}}

文章: {{sentence~}}

{{#block hidden=True}} {{options}}のどれかで回答する必要がある。
{{select "answer" options=options}}
{{/block}}

Answer: {{answer}}
''')

executed_program = program(
    policy_list = policy_list,
    sentence = sentence,
      options=options
)

In [ ]:
# awaitで処理待ち
options = ["Yes", "No", "Maybe"]
policy_list = [
    "メールアドレスが含まれている場合、必ずポリシー違反と判断する。"
]
prompt = guidance('''以下の文章はポリシーに準拠していますか？
ポリシー:
{{#each policy_list}}- {{this}}
{{/each~}}

文章: {{await "sentence"}}

{{#block hidden=True}} {{options}}のどれかで回答する必要がある。
{{select "answer" options=options}} {{/block~}}

Answer: {{answer}}''', stream=True)

# sentenceを与えない場合、sentence以降の処理が進まない
executed_prompt = prompt(
    policy_list = policy_list,
    options=options
)

In [ ]:
prompt2 = executed_prompt(
    sentence="test@gmail.comからの返信です。"
)

# Chat

In [ ]:
import guidance

# モデルの指定
guidance.llm = guidance.llms.OpenAI("gpt-3.5-turbo")

# sysytem, user, assistantを設定
program = guidance('''
{{#system~}}
あなたは情報を探している人を助けるアシスタントです。
{{~/system}}

{{#user~}}
{{conversation_question}}
{{~/user}}

{{#assistant~}}
{{gen 'response' max_tokens=64}}
{{~/assistant}}''')

executed_program = program(conversation_question='ベンサムは誰？')

In [ ]:
system_role = "あなたは日本語を採点するアシスタントです。"

examples = [
    {"role":"user", "content":"ありがとう"},
    {"role":"assistant", "content":"ありがとうは正しいので100点"},
    {"role":"user", "content":"食べれる"},
    {"role":"assistant", "content":"食べられるが正しいので50点"},
]

# 辞書型リストの展開 & ifで分岐
prompt = guidance('''
{{#system~}}
{{system_role}}
{{~/system}}

{{#each examples}}
{{#if (== this.role "user")}}
{{#user}}{{this.content}}{{/user}}
{{else}}
{{#assistant}}{{this.content}}{{/assistant}}
{{/if}}
{{/each}}

{{#user~}}
{{user_input}}
{{~/user}}

{{#assistant~}}
{{gen "response"}}
{{~/assistant}}
''')


# ユーザの入力
user_input = "見れる"

executed_prompt = prompt(
    system_role=system_role,
    examples=examples,
    user_input=user_input
)

In [ ]:
executed_prompt

In [ ]:
import guidance

# モデルの指定
guidance.llm = guidance.llms.OpenAI("gpt-3.5-turbo")

# Few-Shot用のサンプルデータ
sample_data = [
    {"filename": "案件A", "filecontent":"AWSの移行を提案中"},
]

# Few-Shot用サンプルデータを展開するプロンプト
fs_prompt = guidance('''
{{~#each results}}
<result>
{{this.filename}}
{{this.filecontent}}
</result>
{{~/each}}''')

# Few-Shot用のチャット例
examples = [
    {"role":"user", "content":"案件Aの情報を教えて"},
    {"role":"assistant", "content":"<search>案件A</search>"},
    {"role":"user", "content": str(fs_prompt(results=sample_data))},
    {"role":"assistant", "content":"案件Aでは、AWSの移行を提案中です"},
]


# モデルによって実行する関数
def action(query):
    if "search" in query: # モデルがsearchを行うと判断した場合
        # <search>を取り除く
        query = query.split(">")[-1]
        
        # 本番データ
        files = [
             {"filename": "案件A", "filecontent":"AWSの移行を提案中"},
             {"filename": "案件B", "filecontent":"Azureのファイルサーバ受注"},
             {"filename": "施策1", "filecontent":"人材育成コンテンツ作成中"},
        ]

        # queryに該当するものを返す
        results = [file for file in files if file["filename"] == query]
        return results
    
    
# 指示文
instruct = """
あなたは必ず事実に基づいて回答をします。
検索が必要な場合は、機能:<search>query</search>を使うべきです。
私が検索結果をあなたに提示するので、その内容を踏まえて回答して。
"""

# ツールの選択を行い実行するプロンプト
prompt = guidance('''
{{#system~}} あなたは情報を探している人を助けるアシスタントです。 {{~/system}}

{{#user~}} {{instruct}} {{~/user}}

{{#assistant~}} はい、指示に従います。いくつかの例を試しましょう。 {{~/assistant}}

{{#each examples}}
{{#if (== this.role "user")}}
{{#user}} {{this.content}} {{/user}}
{{else}}
{{#assistant}} {{this.content}} {{/assistant}}
{{/if}}
{{/each}}

{{#user~}} {{user_query}} {{~/user}}

{{#assistant~}} 
{{gen "thought" stop="</search>"}}
{{~/assistant}}

{{#user~}}
Results:
{{~#each (action thought)}}
<result>
{{this.filename}}
{{this.filecontent}}
</result>
{{~/each}}
{{~/user}}

{{#assistant~}}
{{gen "answer"}}
{{~/assistant}}
''')


# ユーザの入力
# user_query = "案件Bについて知りたい"
user_query = "施策1はどうなっている？"

executed_prompt = prompt(
    instruct=instruct,
    examples=examples,
    user_query=user_query,
    is_search=is_search,
    action=action
)